# AI Assisted Annotation with OHIF

Zero foot print Dicom viewers are ideal for radiology workflow. 
[Open Health Imaging Foundation (OHIF)](http://ohif.org/) Viewer is an open source, web-based, medical imaging viewer. 
It can be configured to connect to Image Archives that support DicomWeb, 
and offers support for mapping to proprietary API formats. 
OHIF maintained extensions add support for viewing, annotating, 
and reporting on DICOM images in 2D (slices) and 3D (volumes). 
OHIF by it self is a web viewer that requires a PACS on the back end to hold the dicom images. 
You can use either [Dcm4chee](https://github.com/dcm4che/dcm4chee-arc-light/wiki) or [Orthanc](https://www.orthanc-server.com/). 

For this we expand on how to use OHIF with AIAA. 
This note book will guide your through how to set up 
- OHIF web viewer with Nvidia AIAA integration 
- Orthanc
- This current notebooks 


## Prerequisites
- Nvidia GPU with 8Gb of memory
- Going through [AIAA Notebook](AIAA.ipynb) to get basic understanding of AIAA.    
- For trouble shot, basic understanding of docker and docker compose is recommended 


# 0. Licensing

### 0.1 ORTHANC License
This note book will help you set up ORTHANC as a pacs back end to OHIF. 
Please read through the license of ORTHANC and its plugins 
https://book.orthanc-server.com/faq/licensing.html

Please note: ORTHANC has plugins which in turn has more restrictive licenses as 
- Orthanc Core: GPL v3 https://hg.orthanc-server.com/orthanc/file/tip/COPYING
- Web viewer: AGPL v3 https://hg.orthanc-server.com/orthanc-webviewer/file/tip/COPYING
- PostgreSQL Support: AGPL v3 https://hg.orthanc-server.com/orthanc-postgresql/file/tip/COPYING
- DICOMweb Support: AGPL v3 https://hg.orthanc-server.com/orthanc-dicomweb/file/tip/COPYING
- Whole slide imaging: AGPL v3 https://hg.orthanc-server.com/orthanc-wsi/file/tip/COPYING

ORTHANC will be pulled as a docker image as it is specified in the docker compose file
```
  orthanc:
    image: jodogne/orthanc-plugins:1.9.1 # Orthanc docker image with plugins
```
### 0.2 Nginx License
Docker compose will set up OHIF which in turn uses [Nginx](https://nginx.org/en/) 
from their website it states 
"The sources and documentation are distributed under the [2-clause BSD-like license](https://nginx.org/LICENSE)"
   

## 1. Setup OHIF 
Nvidia has integrated AIAA into OHIF as a plugin. 
We have packaged all under OHIF-Orthanc sub-folder. 
This will launch OHIF with Orthanc and this current notebooks.<br>
**Note: You need to close this docker container before doing steps below. 
You can do this by running `scripts/stopClaraTrainNoteBooks.sh`** <br> 
These steps will launch multiple dockers as shown below
<br><img src="screenShots/ThreeDockers.png" alt="Drawing" style="height: 300px;"/><br>

1. Open terminal on your bare bone host
2. cd into `AIAA/OHIF-Orthanc` sub-folders
3. (optional) edit docker-compose.yml to change/ map the data dir. 
By default it will be inside the sub-folder. 
You can simply change the `ConfigLocalPath=./data` in the `.env` file.
4. Run `./restart.sh`. This will launch this notebooks, OHIF, and orthanc
5. Basic authentication is provided.  


### 1.2. Check that everything is running:
Your main landing page is [yourip:3030/start/](http://localhost:3030/start/). 
User/password is `clara/clara`. 
To change this user/password please refer to exercise section below. 
You should see a page as below with links as listed below 
<br><img src="screenShots/OHIFmainPage.png" alt="Drawing" style="height: 300px;"/><br> 

  
1. This Notebooks: </br>
Go to your browser at [yourip:3030/notebooks/](http://localhost:3030/notebooks/) 
and use the token printed out after running the `restart.sh` script
2. OHIF: </br> 
Go to your browser [yourip:3030](http://localhost:3030/) to see OHIF web page as below
<br><img src="screenShots/OHIFList.png" alt="Drawing" style="height: 300px;"/><br> 
3. Orthanc: </br> 
Go to your browser at [yourip:3030/pacs-admin/](http://localhost:3030/pacs-admin) to see Orthanc web UI (username/password is orthanc/orthanc) 
<br><img src="screenShots/Orthanc.png" alt="Drawing" style="height: 300px;"/><br> 

## 2. Download Dicom Studies

OHIF and the back end PACS uses dicom format. 
For this you need to have a some Dicom studies locally.
You can run [TCIA Notebook](../Data/TCIA/TCIADownloader.ipynb) 
with any of the sample data links. 
For basic workflow you can stop right after you download the dicom data and unzipping it. 

You should find the downloaded data under `/claraDevDay/Data/<Dataname>/DCM/`  


## 3. Upload a dicom study into the PACS (orthanc)
Now that you have dicom images, 
we would want to upload it to our PACS. 
For this we can use dcmtk library  

In [ ]:
# install apt-get install dcmtk
!apt-get -y install dcmtk

As we are using docker compose internal network we can directly upload to orthanc using its intenal name 
`orthanc` and its internal port 4242 

In [ ]:
!storescu -v +sd +r -xb -v -aet "fromtest" -aec "ORTHANC" orthanc 4242 /claraDevDay/Data/NSCLC_5/DCM/1.3*

## 4. Start AIAA server 
You should run the [AIAA Notebook](AIAA.ipynb) and start the AIAA server, then load at least one model. 
We recommend the 2D deepgrow model as a start.  

## 5. Open OHIF and use AIAA
Now that we have dicom data in our PACS, you can open OHIF in your browser and click on a patient to view it as below.
<br><img src="screenShots/OHIF.png" alt="Drawing" style="height: 300px;"/><br> 


Next you need to setup the AIAA plugin (see image below) :
1. AIAA server url:
    1. Set to `http://<yourmachineIP>:3030/aiaa/` 
    2. Click refresh icon next to it.
    3. You should see the models you have loaded in AIAA server.
2. Setup fetch from pacs:
    1. Under more AIAA setup
    2. Check Fetch Images From Dicom Server 
    3. Dicom server = `orthanc:4242`. This is the internal port exposed through docker network
    4. Set AETitle = `ORTHANC`
<br><img src="screenShots/OHIF-AIAAwRevProxy.png" alt="Drawing" style="height: 300px;"/><br> 


## 6. Stop All
To stop all dockers (Notebooks, OHIF and PACS) you should run `./stop.sh`. 
***Note: Running cell below would terminate this notebook***


In [ ]:
#Note: Running cell below would terminate this notebook
#!./stop.sh`


# Exercise 

### 1. Remove user/password
In order to have no security at all, you can change the [docker-compose](OHIF-Orthanc/docker-compose.yml) 
to use [nginxNoPassword](OHIF-Orthanc/config/nginxNoPassword.conf) instead of [nginxWPassword](OHIF-Orthanc/config/nginxWPassword.conf) 

### 2. Change user/password file
We provide basic authentication file [`.htpasswd`](OHIF-Orthanc/config/.htpasswd) 
with user/password = clara/clara.
In order to change this or create a new file altogether, you can run 

In [ ]:
htpasswd -c /claraDevDay/AIAA/OHIF-Orthanc/config/.MyNewhtpasswd username1
# you would be prompt for password then again to confirm it 

To a add another user we can use the same command without `-c` flag as 

In [ ]:
htpasswd /claraDevDay/AIAA/OHIF-Orthanc/config/.MyNewhtpasswd anotherusername2 


### 3. Use https instead of http
Setup above is using http for simplicity that would work on internal secure network. 
In order to use this setup on public you should use https to secure communication between client and server. 
For this we would need to:
1. Have a ssl certification and key. For this you can:
    1. Use the sample self certified crt and key files provided  
    2. Use your own ssl files
    3. Generate your own self signed by running 
    ```
    openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout /claraDevDay/AIAA/OHIF-Orthanc/config/nginx-selfsigned.key -out /claraDevDay/AIAA/OHIF-Orthanc/config/nginx-selfsigned.crt
    ```
    For more options you can follow instructions [here](https://www.digitalocean.com/community/tutorials/how-to-create-a-self-signed-ssl-certificate-for-nginx-in-ubuntu-16-04)
2. Change the [docker-compose.yml](OHIF-Orthanc/docker-compose.yml) to:
    1. Map port 443 instead of port 80 
    2. Map the crt and key files.
    3. Use [`nginxHttpsWPassword.conf`](OHIF-Orthanc/config/nginxHttpsWPassword.conf)

If you use self signed certification would would get warnings on your first access as shown below. 
You would need to click `Advanced` then click `Proceed to _____ (unsafe)` link as shown below

<br><img src="screenShots/httpsWarning.png" alt="Drawing" style="height: 300px;"/><br> 
